# Climate change impacts of NG supply-demand scenarios 

In [ ]:
import os
import copy
from collections import namedtuple
import warnings
import IPython.display as ipd
from IPython.display import display, Latex
import string

import pickle
import pprint
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from scipy.stats import norm
from statsmodels.stats import weightstats as stests
from scipy.stats import pearsonr
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import brightway2 as bw

import ng_lca_toolbox
from ng_lca_toolbox import (
    excel2dfs,
    excel2ngScenarioLCA,
    plot_ipcc_breakdown,
    plot_cc_hist,
    plot_cc_violin,
    plot_cc_hist_comparison,
)

warnings.filterwarnings("ignore")

LCAResult = namedtuple("LCAResult", ["demand", "method", "score"])

COLORS = ["#66c2a5", "#fc8d62", "#ffd92f", "#8da0cb", "#e78ac3"]
SCENARIOS = ["Base", "REPowerEU", "Aftermath", "Coal", "Clean"]
supply_scenario_y2021, supply_scenario_y2022, supply_scenario_alternative = (
    "y2021_mid",
    "y2022_mid",
    "alternative_mid",
)

pprinter = pprint.PrettyPrinter(indent=4).pprint
plt.rcParams["svg.fonttype"] = "path"

BW_PROJECT = "NG_2022crisis_LCA"
DB_NAME = "ecoinvent 3.9.1 cutoff, natural gas scenario EU27"

bw.projects.set_current(BW_PROJECT)  # Accessing the project
ei_ng_db = bw.Database(DB_NAME)

RUN_SCENARIOS = False
SKIP_LCA = []  # ["Base", "REPowerEU", "Aftermath", "Coal"]#

save_fig = False
save_tables = False
if os.path.exists("./data/results") is False:
    os.makedirs("./data/results")

PATH2PICKLES = "./data/persisted_files/"
if os.path.isdir(PATH2PICKLES) is False:
    os.makedirs(PATH2PICKLES)

RUN_MC = False
SKIP_MC = []  # ["Base", "REPowerEU", "Coal", "Clean"]
N_mc = 1000
MC_PATH = f"./data/persisted_files/monte_carlo_{N_mc}_ite.pickle"
MC_PATH_LCA_ONLY = f"./data/persisted_files/monte_carlo_LCA_only.pickle"

fig_length = {
    1: 3.50394,  # 1 column
    1.5: 5.35433,  # 1.5 columns
    2: 7.20472,
}  # 2 columns
fig_height = 9.72441  # maxium height

## Scenarios' LCIA

### Base-REPowerEU comparison

In [ ]:
# Scenarios IPCC analysis
# Base scenario:
if (
    RUN_SCENARIOS or not os.path.exists(PATH2PICKLES + "scenarios_ipcc_repower.pickle")
) and "Base" not in SKIP_LCA:
    baseScenario = excel2ngScenarioLCA("base", supply_scenario_y2021, ei_ng_db)
    repowerScenario = excel2ngScenarioLCA(
        "repower", supply_scenario_alternative, ei_ng_db
    )

    # base with new NG/LNG mix
    rep_new_ng_mix = copy.deepcopy(baseScenario)
    rep_new_ng_mix.NG_DE = repowerScenario.NG_DE
    rep_new_ng_mix.NG_NL = repowerScenario.NG_NL
    rep_new_ng_mix.NG_RO = repowerScenario.NG_RO
    rep_new_ng_mix.NG_NO = repowerScenario.NG_NO
    rep_new_ng_mix.NG_RU = repowerScenario.NG_RU
    rep_new_ng_mix.NG_DZ = repowerScenario.NG_DZ
    rep_new_ng_mix.NG_AZ = repowerScenario.NG_AZ
    rep_new_ng_mix.LNG_US = repowerScenario.LNG_US
    rep_new_ng_mix.LNG_QA = repowerScenario.LNG_QA
    rep_new_ng_mix.LNG_RU = repowerScenario.LNG_RU
    rep_new_ng_mix.LNG_NG = repowerScenario.LNG_NG
    rep_new_ng_mix.LNG_DZ = repowerScenario.LNG_DZ

    # REPowerEU: coal
    rep_coal = copy.deepcopy(rep_new_ng_mix)
    rep_coal.NG_POWER -= repowerScenario.COAL_POWER
    rep_coal.COAL_POWER = repowerScenario.COAL_POWER

    # REPowerEU: coal + nuclear
    rep_coal_nuc = copy.deepcopy(rep_coal)
    rep_coal_nuc.NG_POWER -= repowerScenario.NUCLEAR_POWER
    rep_coal_nuc.NUCLEAR_POWER = repowerScenario.NUCLEAR_POWER

    # REPowerEU: coal + nuclear + electric heaters
    rep_coal_nuc_heatpump = copy.deepcopy(rep_coal_nuc)
    rep_coal_nuc_heatpump.NG_HOUSEHOLDS -= repowerScenario.ELECTRICITY_HOUSEHOLDS
    rep_coal_nuc_heatpump.ELECTRICITY_HOUSEHOLDS = (
        repowerScenario.ELECTRICITY_HOUSEHOLDS
    )

In [ ]:
# IPCC results of all measures from REPowerEU
ipcc_method = [
    met
    for met in list(bw.methods)
    if met[0] == "IPCC 2021"
    and "climate change: including SLCFs" in met[1]
    and ("GWP100" in met[2] or "GWP20" in met[2])
]

if (
    RUN_SCENARIOS or not os.path.exists(PATH2PICKLES + "scenarios_ipcc_repower.pickle")
) and "Base" not in SKIP_LCA:
    scenarios_ipcc_repower = [
        baseScenario,
        rep_new_ng_mix,
        rep_coal,
        rep_coal_nuc,
        rep_coal_nuc_heatpump,
        repowerScenario,
    ]

    [sce.doLCA(ipcc_method) for sce in scenarios_ipcc_repower]

    for sce in scenarios_ipcc_repower:
        sce.lca_results = [
            LCAResult(
                demand=l_.demand,
                method=l_.method,
                score=l_.score,
            )
            for l_ in sce.lca_results
        ]

    with open(PATH2PICKLES + "scenarios_ipcc_repower.pickle", "wb") as fp:
        pickle.dump(scenarios_ipcc_repower, fp)
else:
    with open(PATH2PICKLES + "scenarios_ipcc_repower.pickle", "rb") as fp:
        scenarios_ipcc_repower = pickle.load(fp)

ipcc_res_repower = [sce.lca_results[0].score / 1e12 for sce in scenarios_ipcc_repower]
print(ipcc_res_repower)
ipcc_res2 = [
    (ipcc_ - ipcc_res_repower[i]) for i, ipcc_ in enumerate(ipcc_res_repower[1::])
]

In [ ]:
scenario_names_repower = [
    "Base (year 2021)",
    "New supply mix",
    "(+) Coal",
    "(+) Nuclear",
    "(+) Heat pumps",
    "(+) Savings household heat",
    "REPowerEU (short-term)",
]
color_scenario_repower = "#fc8d62"
assert len(scenario_names_repower) == len(scenarios_ipcc_repower) + 1
fig, ax = plot_ipcc_breakdown(
    scenarios_ipcc_repower,
    scenario_names_repower,
    color_scenario_repower,
    save_fig=save_fig,
    path2save="./figs/ipcc_repower",
)

ipd.clear_output()
plt.show()

### Base-Coal comparison

In [ ]:
# Scenarios IPCC analysis - Coal
# Base scenario:
if (
    RUN_SCENARIOS or not os.path.exists(PATH2PICKLES + "scenarios_ipcc_coal.pickle")
) and SCENARIOS[3] not in SKIP_LCA:
    baseScenario2 = excel2ngScenarioLCA("base", supply_scenario_y2021, ei_ng_db)
    coalScenario2 = excel2ngScenarioLCA("coal", supply_scenario_alternative, ei_ng_db)

    # base with new NG/LNG mix
    coal_new_ng_mix = copy.deepcopy(baseScenario2)
    coal_new_ng_mix.NG_DE = coalScenario2.NG_DE
    coal_new_ng_mix.NG_NL = coalScenario2.NG_NL
    coal_new_ng_mix.NG_RO = coalScenario2.NG_RO
    coal_new_ng_mix.NG_NO = coalScenario2.NG_NO
    coal_new_ng_mix.NG_RU = coalScenario2.NG_RU
    coal_new_ng_mix.NG_DZ = coalScenario2.NG_DZ
    coal_new_ng_mix.NG_AZ = coalScenario2.NG_AZ
    coal_new_ng_mix.LNG_US = coalScenario2.LNG_US
    coal_new_ng_mix.LNG_QA = coalScenario2.LNG_QA
    coal_new_ng_mix.LNG_RU = coalScenario2.LNG_RU
    coal_new_ng_mix.LNG_NG = coalScenario2.LNG_NG
    coal_new_ng_mix.LNG_DZ = coalScenario2.LNG_DZ

    # REPowerEU: coal
    coal_coal2 = copy.deepcopy(coal_new_ng_mix)
    coal_coal2.NG_POWER -= coalScenario2.COAL_POWER
    coal_coal2.COAL_POWER = coalScenario2.COAL_POWER

In [ ]:
# IPCC results of all measures from REPowerEU
ipcc_method = [
    met
    for met in list(bw.methods)
    if met[0] == "IPCC 2021"
    and "climate change: including SLCFs" in met[1]
    and ("GWP100" in met[2] or "GWP20" in met[2])
]

if (
    RUN_SCENARIOS or not os.path.exists(PATH2PICKLES + "scenarios_ipcc_coal.pickle")
) and SCENARIOS[3] not in SKIP_LCA:
    scenarios_ipcc_coal = [baseScenario2, coal_new_ng_mix, coal_coal2, coalScenario2]

    [sce.doLCA(ipcc_method) for sce in scenarios_ipcc_coal]

    for sce in scenarios_ipcc_coal:
        sce.lca_results = [
            LCAResult(
                demand=l_.demand,
                method=l_.method,
                score=l_.score,
            )
            for l_ in sce.lca_results
        ]

    with open(PATH2PICKLES + "scenarios_ipcc_coal.pickle", "wb") as fp:
        pickle.dump(scenarios_ipcc_coal, fp)
else:
    with open(PATH2PICKLES + "scenarios_ipcc_coal.pickle", "rb") as fp:
        scenarios_ipcc_coal = pickle.load(fp)

ipcc_res_coal = [sce.lca_results[0].score / 1e12 for sce in scenarios_ipcc_coal]
print(ipcc_res_coal)
ipcc_res2 = [(ipcc_ - ipcc_res_coal[i]) for i, ipcc_ in enumerate(ipcc_res_coal[1::])]

In [ ]:
# plot bars Coal
scenario_names_coal = [
    "Base (year 2021)",
    "New supply mix",
    "(+) Coal power",
    "Coal scenario",
]
color_scenario_coal = "#8da0cb"
assert len(scenario_names_coal) == len(scenarios_ipcc_coal)
fig, ax = plot_ipcc_breakdown(
    scenarios_ipcc_coal[:-1],
    scenario_names_coal,
    color_scenario_coal,
    save_fig=save_fig,
    path2save="./figs/ipcc_coal",
)

ipd.clear_output()
plt.show()

### Base-Clean comparison

In [ ]:
# Scenarios IPCC analysis - Clean
# Base scenario:
if (RUN_SCENARIOS or not os.path.exists(PATH2PICKLES + "scenarios_ipcc_clean.pickle")) and SCENARIOS[4] not in SKIP_LCA:
    baseScenario3 = excel2ngScenarioLCA("base", supply_scenario_y2021, ei_ng_db)
    cleanScenario3 = excel2ngScenarioLCA("clean", supply_scenario_alternative, ei_ng_db)

    # base with new NG/LNG mix
    clean_new_ng_mix = copy.deepcopy(baseScenario3)
    clean_new_ng_mix.NG_DE = cleanScenario3.NG_DE
    clean_new_ng_mix.NG_NL = cleanScenario3.NG_NL
    clean_new_ng_mix.NG_RO = cleanScenario3.NG_RO
    clean_new_ng_mix.NG_NO = cleanScenario3.NG_NO
    clean_new_ng_mix.NG_RU = cleanScenario3.NG_RU
    clean_new_ng_mix.NG_DZ = cleanScenario3.NG_DZ
    clean_new_ng_mix.NG_AZ = cleanScenario3.NG_AZ
    clean_new_ng_mix.LNG_US = cleanScenario3.LNG_US
    clean_new_ng_mix.LNG_QA = cleanScenario3.LNG_QA
    clean_new_ng_mix.LNG_RU = cleanScenario3.LNG_RU
    clean_new_ng_mix.LNG_NG = cleanScenario3.LNG_NG
    clean_new_ng_mix.LNG_DZ = cleanScenario3.LNG_DZ

    # REPowerEU: solar
    clean_solar3 = copy.deepcopy(clean_new_ng_mix)
    clean_solar3.NG_POWER -= cleanScenario3.SOLAR_POWER
    clean_solar3.SOLAR_POWER = cleanScenario3.SOLAR_POWER

    # REPowerEU: wind
    clean_wind = copy.deepcopy(clean_solar3)
    clean_wind.NG_POWER -= cleanScenario3.WIND_POWER
    clean_wind.WIND_POWER = cleanScenario3.WIND_POWER

In [ ]:
# IPCC results of all measures from REPowerEU
ipcc_method = [
    met
    for met in list(bw.methods)
    if met[0] == "IPCC 2021"
    and "climate change: including SLCFs" in met[1]
    and ("GWP100" in met[2] or "GWP20" in met[2])
]

if (RUN_SCENARIOS or not os.path.exists(PATH2PICKLES + "scenarios_ipcc_clean.pickle")) and SCENARIOS[3] not in SKIP_LCA:
    scenarios_ipcc_clean = [
        baseScenario3,
        clean_new_ng_mix,
        clean_solar3,
        clean_wind,
        cleanScenario3,
    ]

    [sce.doLCA(ipcc_method) for sce in scenarios_ipcc_clean]

    for sce in scenarios_ipcc_clean:
        sce.lca_results = [
            LCAResult(
                demand=l_.demand,
                method=l_.method,
                score=l_.score,
            )
            for l_ in sce.lca_results
        ]

    with open(PATH2PICKLES + "scenarios_ipcc_clean.pickle", "wb") as fp:
        pickle.dump(scenarios_ipcc_clean, fp)
else:
    with open(PATH2PICKLES + "scenarios_ipcc_clean.pickle", "rb") as fp:
        scenarios_ipcc_clean = pickle.load(fp)

ipcc_res_clean = [sce.lca_results[0].score / 1e12 for sce in scenarios_ipcc_clean]
print(ipcc_res_clean)
ipcc_res_clean2 = [
    (ipcc_ - ipcc_res_clean[i]) for i, ipcc_ in enumerate(ipcc_res_clean[1::])
]

In [ ]:
# plot bars Clean
scenario_names_clean = [
    "Base (year 2021)",
    "New supply mix",
    "(+) Solar power",
    "(+) Wind power",
    "(+) Household heat savings",
    "Clean scenario",
]
color_scenario_clean = "#e78ac3"
assert len(scenario_names_clean) == len(scenarios_ipcc_clean) + 1
fig, ax = ng_lca_toolbox.plot_ipcc_breakdown(
    scenarios_ipcc_clean,
    scenario_names_clean,
    color_scenario_clean,
    save_fig=save_fig,
    path2save="./figs/ipcc_clean",
)

ipd.clear_output()
plt.show()

### Base-y2022 comparison

In [ ]:
# Scenarios IPCC analysis - Clean
# Base scenario:
if (RUN_SCENARIOS or not os.path.exists(PATH2PICKLES + "scenarios_ipcc_y2022.pickle")) and SCENARIOS[2] not in SKIP_LCA:
    baseScenario4 = ng_lca_toolbox.excel2ngScenarioLCA("base", supply_scenario_y2021, ei_ng_db)
    y2022Scenario4 = ng_lca_toolbox.excel2ngScenarioLCA("y2022", supply_scenario_y2022, ei_ng_db)

    # base with new NG/LNG mix
    y2022_new_ng_mix = copy.deepcopy(baseScenario4)
    y2022_new_ng_mix.NG_DE = y2022Scenario4.NG_DE
    y2022_new_ng_mix.NG_NL = y2022Scenario4.NG_NL
    y2022_new_ng_mix.NG_RO = y2022Scenario4.NG_RO
    y2022_new_ng_mix.NG_NO = y2022Scenario4.NG_NO
    y2022_new_ng_mix.NG_RU = y2022Scenario4.NG_RU
    y2022_new_ng_mix.NG_DZ = y2022Scenario4.NG_DZ
    y2022_new_ng_mix.NG_AZ = y2022Scenario4.NG_AZ
    y2022_new_ng_mix.LNG_US = y2022Scenario4.LNG_US
    y2022_new_ng_mix.LNG_QA = y2022Scenario4.LNG_QA
    y2022_new_ng_mix.LNG_RU = y2022Scenario4.LNG_RU
    y2022_new_ng_mix.LNG_NG = y2022Scenario4.LNG_NG
    y2022_new_ng_mix.LNG_DZ = y2022Scenario4.LNG_DZ

    # y2022: + coal
    y2022_coal = copy.deepcopy(y2022_new_ng_mix)
    y2022_coal.NG_POWER -= y2022Scenario4.COAL_POWER
    y2022_coal.COAL_POWER = y2022Scenario4.COAL_POWER
    
    # y2022: - Nuclear
    y2022_NUCLEAR = copy.deepcopy(y2022_coal)
    y2022_NUCLEAR.NUCLEAR_POWER = max(y2022Scenario4.NUCLEAR_POWER, 0)
    y2022_NUCLEAR.NG_POWER -= y2022Scenario4.NUCLEAR_POWER
    
    # y2022: + WIND
    y2022_WIND = copy.deepcopy(y2022_NUCLEAR)
    y2022_WIND.NG_POWER -= y2022Scenario4.WIND_POWER
    y2022_WIND.WIND_POWER = y2022Scenario4.WIND_POWER

    # y2022: + solar
    y2022_SOLAR = copy.deepcopy(y2022_WIND)
    y2022_SOLAR.NG_POWER -= y2022Scenario4.SOLAR_POWER
    y2022_SOLAR.SOLAR_POWER = y2022Scenario4.SOLAR_POWER
    
    # y2022: - HYDRO
    y2022_HYDRO = copy.deepcopy(y2022_SOLAR)
    y2022_HYDRO.HYDRO_POWER = max(y2022Scenario4.HYDRO_POWER, 0)
    y2022_HYDRO.NG_POWER -= y2022Scenario4.HYDRO_POWER
    
    # y2022: + SAVINGS POWER
    y2022_SAVINGS = copy.deepcopy(y2022_HYDRO)
    y2022_SAVINGS.NG_POWER -= y2022Scenario4.SAVINGS_POWER
    y2022_SAVINGS.SAVINGS_POWER = y2022Scenario4.SAVINGS_POWER

    # INDUSTRIAL HEAT
    # y2022: + OIL
    y2022_OIL = copy.deepcopy(y2022_SAVINGS)
    y2022_OIL.NG_INDUSTRY -= y2022Scenario4.OIL_INDUSTRY
    y2022_OIL.OIL_INDUSTRY = y2022Scenario4.OIL_INDUSTRY
    
    # INDUSTRIAL HEAT
    # y2022: + EFF_IND
    y2022_EFF_IND = copy.deepcopy(y2022_OIL)
    y2022_EFF_IND.NG_INDUSTRY -= y2022Scenario4.EFFICIENCY_INDUSTRY
    y2022_EFF_IND.EFFICIENCY_INDUSTRY = y2022Scenario4.EFFICIENCY_INDUSTRY
    
    # y2022: + EFF_IND
    y2022_SAVS_IND = copy.deepcopy(y2022_EFF_IND)
    y2022_SAVS_IND.NG_INDUSTRY -= y2022Scenario4.SAVINGS_INDUSTRY
    y2022_SAVS_IND.SAVINGS_INDUSTRY = y2022Scenario4.SAVINGS_INDUSTRY   
    
    # HOUSEHOLD HEAT 
    # y2022: + ELECTRICITY_HOUSEHOLDS
    y2022_ELEC_HOUSE = copy.deepcopy(y2022_SAVS_IND)
    y2022_ELEC_HOUSE.NG_HOUSEHOLDS -= y2022Scenario4.ELECTRICITY_HOUSEHOLDS
    y2022_ELEC_HOUSE.ELECTRICITY_HOUSEHOLDS = y2022Scenario4.ELECTRICITY_HOUSEHOLDS   
    
    # y2022: + WEATHER_HOUSEHOLDS
    y2022_WEATHER_HOUSE = copy.deepcopy(y2022_ELEC_HOUSE)
    y2022_WEATHER_HOUSE.NG_HOUSEHOLDS -= y2022Scenario4.WEATHER_HOUSEHOLDS
    y2022_WEATHER_HOUSE.WEATHER_HOUSEHOLDS = y2022Scenario4.WEATHER_HOUSEHOLDS   
    
    # y2022: + EFFICIENCY_HOUSEHOLDS
    y2022_EFF_HOUSE = copy.deepcopy(y2022_WEATHER_HOUSE)
    y2022_EFF_HOUSE.NG_HOUSEHOLDS -= y2022Scenario4.EFFICIENCY_HOUSEHOLDS
    y2022_EFF_HOUSE.EFFICIENCY_HOUSEHOLDS = y2022Scenario4.EFFICIENCY_HOUSEHOLDS
    
    # y2022: + SAVINGS_HOUSEHOLDS
    y2022_SAVINGS_HOUSE = copy.deepcopy(y2022_EFF_HOUSE)
    y2022_SAVINGS_HOUSE.NG_HOUSEHOLDS -= y2022Scenario4.SAVINGS_HOUSEHOLDS
    y2022_SAVINGS_HOUSE.SAVINGS_HOUSEHOLDS = y2022Scenario4.SAVINGS_HOUSEHOLDS  

    # IPCC results of all measures    
    y2022Scenario4.NUCLEAR_POWER = max(y2022Scenario4.NUCLEAR_POWER, 0.0)
    y2022Scenario4.HYDRO_POWER = max(y2022Scenario4.HYDRO_POWER, 0.0)

In [ ]:
# IPCC results of all measures from REPowerEU
ipcc_method = [
    met
    for met in list(bw.methods)
    if met[0] == "IPCC 2021"
    and "climate change: including SLCFs" in met[1]
    and ("GWP100" in met[2] or "GWP20" in met[2])
]

if (RUN_SCENARIOS or not os.path.exists(PATH2PICKLES + "scenarios_ipcc_y2022.pickle")) and SCENARIOS[2] not in SKIP_LCA:
    scenarios_ipcc_y2022 = [
        baseScenario4,
        y2022_new_ng_mix,
        y2022_coal,
        y2022_NUCLEAR,
        y2022_WIND,
        y2022_SOLAR,
        y2022_HYDRO,
        y2022_SAVINGS,
        y2022_OIL,
        y2022_EFF_IND,
        y2022_SAVS_IND,
        y2022_ELEC_HOUSE,
        y2022_WEATHER_HOUSE,
        y2022_EFF_HOUSE,
        y2022_SAVINGS_HOUSE,
        y2022Scenario4,
    ]

    [sce.doLCA(ipcc_method) for sce in scenarios_ipcc_y2022]

    for sce in scenarios_ipcc_y2022:
        sce.lca_results = [
            LCAResult(
                demand=l_.demand,
                method=l_.method,
                score=l_.score,
            )
            for l_ in sce.lca_results
        ]

    with open(PATH2PICKLES + "scenarios_ipcc_y2022.pickle", "wb") as fp:
        pickle.dump(scenarios_ipcc_y2022, fp)
else:
    with open(PATH2PICKLES + "scenarios_ipcc_y2022.pickle", "rb") as fp:
        scenarios_ipcc_y2022 = pickle.load(fp)

ipcc_resy2022 = [sce.lca_results[0].score / 1e12 for sce in scenarios_ipcc_y2022]
print(ipcc_resy2022)
ipcc_resy20222 = [
    (ipcc_ - ipcc_resy2022[i]) for i, ipcc_ in enumerate(ipcc_resy2022[1::])
]

In [ ]:
# plot bars - y2022
scenario_names_y2022 = [
    "Base (year 2021)",
    "New supply mix",
    "(+) Coal",
    "(-) Nuclear",
    "(+) Wind",
    "(+) Solar",
    "(-) Hydro",
    "(+) Savings",
    "(+) Oil",
    "(+) Efficiency Industry",
    "(+) Savings Industry",
    "(+) Heat pumps",
    "(+) Mild winter",
    "(+) Efficiency household",
    "(+) Savings household",
    "Aftermath (year 2022)",
]
color_scenario_y2022 = "#ffd92f"
assert len(scenario_names_y2022) == len(scenarios_ipcc_y2022)
fig, ax = plot_ipcc_breakdown(
    scenarios_ipcc_y2022[:-1],
    scenario_names_y2022,
    color_scenario_y2022,
    save_fig=save_fig,
    path2save="./figs/ipcc_y2022",
)

ipd.clear_output()
plt.show()

## Absolute climate change assessment

In [ ]:
# Runing IPCC CO2-only for the 4 scenarios
ipcc_method_100 = [
    met
    for met in list(bw.methods)
    if met[0] == "IPCC 2021"
    and "climate change: including SLCFs" in met[1]
    and ("GWP100" in met[2])
]

IPCC_CO2_ONLY = ("IPCC 2021", "Life cycle CO2 emissions")
try:
    bw.Method(IPCC_CO2_ONLY).load()
except:
    gwp = bw.Method(ipcc_method_100[0]).load()
    co2_cf = [
        i
        for i in gwp
        if "Carbon dioxide" in bw.Database("biosphere3").get(i[0][1])["name"]
    ]
    metadata = {"unit": "kg CO2"}
    co2_method = bw.Method(("IPCC 2021", "Life cycle CO2 emissions"))
    co2_method.register(**metadata)
    co2_method.write(co2_cf)
    co2_method.process()

if RUN_SCENARIOS or not os.path.exists(PATH2PICKLES + "scenarios_co2_only.pickle"):
    scenarios_co2_only = [
        excel2ngScenarioLCA("base", supply_scenario_y2021, ei_ng_db),
        excel2ngScenarioLCA("repower", supply_scenario_alternative, ei_ng_db),
        excel2ngScenarioLCA("y2022", supply_scenario_y2022, ei_ng_db),
        excel2ngScenarioLCA("coal", supply_scenario_alternative, ei_ng_db),
        excel2ngScenarioLCA("clean", supply_scenario_alternative, ei_ng_db),
    ]
    scenarios_co2_only[2].HYDRO_POWER = max(scenarios_co2_only[2].HYDRO_POWER, 0.0)
    scenarios_co2_only[2].NUCLEAR_POWER = max(scenarios_co2_only[2].NUCLEAR_POWER, 0.0)

    [sce.doLCA([IPCC_CO2_ONLY]) for sce in scenarios_co2_only]

    for sce in scenarios_co2_only:
        sce.lca_results = [
            LCAResult(
                demand=l_.demand,
                method=l_.method,
                score=l_.score,
            )
            for l_ in sce.lca_results
        ]

    with open(PATH2PICKLES + "scenarios_co2_only.pickle", "wb") as fp:
        pickle.dump(scenarios_co2_only, fp)
else:
    with open(PATH2PICKLES + "scenarios_co2_only.pickle", "rb") as fp:
        scenarios_co2_only = pickle.load(fp)

ipcc_methods = [" ".join(met) for met in scenarios_co2_only[0].lca_methods]
dict_of_ipccs = {k: [] for k in [IPCC_CO2_ONLY]}

for sce in scenarios_co2_only:
    for i, met in enumerate([IPCC_CO2_ONLY]):
        dict_of_ipccs[met].append(sce.lca_results[i].score)

In [ ]:
# Calculation budget for 1.5 and 2°C temperature targets
df_ipcc = pd.DataFrame(
    np.array([list(dict_of_ipccs.values())[0] for k in ["1_5C", "2C"]]).T,
    columns=["1_5C", "2C"],
    # index=[f'scenario_{i}' for i in range(len(scenarios2))],
    index=SCENARIOS,
)


EU_POPULATION = 447.0 * 1e6
GLOBAL_POPULATION = 7.9 * 1e9
CUM_GLOBAL_POPULATION = 799_098_891_918 # from 2020-2100 https://www.sciencedirect.com/science/article/abs/pii/S0959378017312153?via%3Dihub
cum_budget_1_5C = 400e12 # 66.7% likelihood https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_SPM_Stand_Alone.pdf = 501 kg CO2
cum_budget_2C = 1150e12 # 66.7% likelihood https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_SPM_Stand_Alone.pdf = 1440.375 kg CO2

# constant yearly budget:
df_ipcc["1_5C_cte_year_budget"] = df_ipcc["1_5C"] / (cum_budget_1_5C / 80 * (EU_POPULATION / GLOBAL_POPULATION))
df_ipcc["2C_cte_year_budget"] = df_ipcc["2C"] / (cum_budget_2C / 80 * (EU_POPULATION / GLOBAL_POPULATION))

# Data from IPCC budget to temperature goal of 1.5 C with 66.7 % chance
df_ipcc["1_5C"] = df_ipcc["1_5C"] / (cum_budget_1_5C * EU_POPULATION / CUM_GLOBAL_POPULATION)
# Data from IPCC budget to temperature goal of 2.0 C with 66.7 % chance
df_ipcc["2C"] = df_ipcc["2C"] / (cum_budget_2C * EU_POPULATION / CUM_GLOBAL_POPULATION)

df_ipcc

In [ ]:
# Plot carbon budget results in polar axis
COLORS = [
    "#66c2a5",
    color_scenario_repower,
    color_scenario_y2022,
    color_scenario_coal,
    color_scenario_clean,
]


def make_barpolar(vals, labels=None, colors=None, layout_options=None, **fig_kwargs):
    # infer slice angles
    num_slices = len(vals)
    theta = [(i + 0.5) * 90 / num_slices for i in range(num_slices)]
    width = [0.5 * (90 / num_slices) for _ in range(num_slices)]

    # optionally infer colors
    if colors is None:
        color_seq = px.colors.qualitative.Safe
        color_indices = range(0, len(color_seq), len(color_seq) // num_slices)
        colors = [color_seq[i] for i in color_indices]

    if layout_options is None:
        layout_options = {}

    if labels is None:
        labels = ["" for _ in range(num_slices)]
        layout_options["showlegend"] = False

    # make figure
    barpolar_plots = [
        go.Barpolar(
            r=[r],
            theta=[t],
            width=[w],
            # error_y=dict(type='data', array=[1,]),
            name=n,
            marker_color=[c],
            marker_line_color="black",
            marker_line_width=0.5,
            **fig_kwargs
        )
        for r, t, w, n, c in zip(vals, theta, width, labels, colors)
    ]

    return barpolar_plots


def make_figures_barploar(
    barpolar_plots, labels=None, colors=None, layout_options=None, **fig_kwargs
):
    num_slices = len(barpolar_plots)

    fig = make_subplots(
        rows=1,
        cols=num_slices,
        specs=[[{"type": "barpolar"}, {"type": "barpolar"}]],
        # subplot_titles=ipcc_methods
    )
    [fig.append_trace(bp, row=1, col=1) for bp in barpolar_plots[0]]
    [fig.append_trace(bp, row=1, col=2) for bp in barpolar_plots[1]]

    # align background
    angular_tickvals = [0.0] + [(i + 0.5) * 90 / 4 for i in range(4)] + [90]
    fig.update_layout(polar_angularaxis_tickvals=angular_tickvals)

    # additional layout parameters
    fig.update_layout(**layout_options)

    return fig


layout_options = {
    "title_x": 0.5,
    #   "title_y": 1.0,
    "showlegend": False,
    #   'width': 4322.832/3 * 0.5* 1.1,
    #   'height': 2917.32/4 * 0.5* 1.1,
    "width": 7.20472 * 300 / 4 * 0.95,
    "height": 9.72441 * 300 / 6,
    "paper_bgcolor": "white",
    "plot_bgcolor": "white",
    "margin_l": 60,
    "margin_r": 60,
    "margin_b": 60,
    "margin_t": 60,
    "font_color": "black",
    "font_family": "arial",
    "font_size": 10,
    "title_pad_b": 50,
    "title_pad_l": 50,
    "title_pad_r": 50,
    "title_pad_t": 50,
}


barplor_data = make_barpolar(
    100 * df_ipcc[df_ipcc.columns.to_list()[0]].values,
    labels=df_ipcc.index.to_list(),
    layout_options=layout_options,
    opacity=1.0,
    colors=COLORS,
)
barplor_data2 = make_barpolar(
    100 * df_ipcc[df_ipcc.columns.to_list()[1]].values,
    labels=df_ipcc.index.to_list(),
    layout_options=layout_options,
    opacity=1.0,
    colors=COLORS,
)


fig_polar = make_figures_barploar(
    [barplor_data, barplor_data2],
    labels=[" "] + df_ipcc.index.to_list() + [" "],
    layout_options=layout_options,
    opacity=1.0,
    colors=COLORS,
)


for f in [fig_polar]:
    f.update_polars(
        angularaxis=dict(direction="clockwise", autotypenumbers="strict"),
        sector=[-0.0, 90],
        angularaxis_nticks=0,
        angularaxis_tickvals=[0.0]
        + [(i + 0.5) * 90 / len(COLORS) for i in range(len(COLORS))]
        + [90],
        angularaxis_ticktext=[" "] + df_ipcc.index.to_list() + [" "],
        radialaxis_tickvals=[0, 100, 200, 300, 400, 500],
        radialaxis_ticksuffix=" %",
        radialaxis_range=[0, 500],
        # radialaxis_title_text='Planetary boundary allocation',
        angularaxis_gridcolor="grey",
        radialaxis_gridcolor="grey",
        bgcolor="white",
        angularaxis_ticks="outside",
        radialaxis_ticks="outside",
        radialaxis_linecolor="white",
        angularaxis_linecolor="black",
        angularaxis_linewidth=0.3,
        radialaxis_linewidth=0.0,
        angularaxis_showgrid=False,
    )
    f.update_xaxes(
        title_text="xaxis 1 title",
        minor_ticks="outside",
        minor_tickcolor="white",
        minor_ticklen=10,
        row=1,
        col=1,
    )
fig_polar.show()

if save_fig:
    fig_polar.write_image("./figs/PB_ipcc_all.svg")

## Uncertainty Analysis

#### Calculation

In [ ]:
# Monte Carlo calculation
# Implementa LANCA land use method - erosion potential
if len([met for met in list(bw.methods) if "LANCA" in met[0]]) == 0:
    metadata = (
        ("LANCA v2.5 - land use", "erosion potential"),
        "kg soil loss",
        "LANCA v2.5 - Characterization Factors for Erosion Potential due to land occupation and transformation (to/from) \
                Available at: https://www.bookshop.fraunhofer.de/buch/LANCA/244600",
        Path("./data/Soil-Erosion-Potential_CFs_LANCA_v2.5.xlsx"),
    )
    lanca_method = bw.ExcelLCIAImporter(
        filepath=metadata[-1],
        name=metadata[0],
        unit=metadata[1],
        description=metadata[2],
    )
    lanca_method.apply_strategies()
    lanca_method.drop_unlinked()
    lanca_method.write_methods(overwrite=True, verbose=True)

all_methods = (
    ipcc_method
    + [IPCC_CO2_ONLY]
    + [
        met
        for met in list(bw.methods)
        if "EF v3.1 no LT" in met[0]
        and (
            "acidification no LT" == met[1]
            or "climate change no LT" == met[1]
            or "ecotoxicity: freshwater no LT" == met[1]
            or "energy resources: non-renewable no LT" == met[1]
            or "eutrophication" in met[1]
            or "human toxicity: carcinogenic no LT" in met[1]
            or "human toxicity: non-carcinogenic no LT" in met[1]
            or "ionising radiation: human health no LT" in met[1]
            or "land use no LT" == met[1]
            or "material resources: metals/minerals no LT" in met[1]
            or "ozone depletion no LT" in met[1]
            or "particulate matter formation no LT" in met[1]
            or "photochemical" in met[1]
            or "water use no LT" in met[1]
        )
    ]
    + [met for met in list(bw.methods) if "LANCA" in met[0]]
)

all_methods_names = [" | ".join(met) for met in all_methods]

dict_ipcc_methods = {
    k: v
    for k, v in zip(
        all_methods_names,
        all_methods,
    )
}

if RUN_MC or not os.path.exists(MC_PATH):
    baseScenario_mc = copy.deepcopy(scenarios_ipcc_repower[0])
    repowerScenario_mc = copy.deepcopy(scenarios_ipcc_repower[-1])
    y2022Scenario_mc = scenarios_ipcc_y2022[-1]
    coalScenario_mc = scenarios_ipcc_coal[-1]
    cleanScenario_mc = scenarios_ipcc_clean[-1]

    list_of_NgScenarioLCA = [
        baseScenario_mc,
        repowerScenario_mc,
        y2022Scenario_mc,
        coalScenario_mc,
        cleanScenario_mc,
    ]

    if len(SKIP_MC) > 0:
        try:
            with open(MC_PATH, "rb") as pf:
                mc_lca = pickle.load(pf)
        except FileNotFoundError:
            raise FileNotFoundError(
                "No Monte Carlo results found. Please set RUN_MC to True."
            )
    else:
        mc_lca = dict()
    for sce_name, sce in zip(SCENARIOS, list_of_NgScenarioLCA):
        if sce_name in SKIP_MC:
            continue
        print(f"Running Monte Carlo for {sce_name}")
        mc_lca[sce_name] = sce.multi_lcia_MonteCarlo(
            N_mc,
            dict_ipcc_methods,
            seed=42,
        )
else:
    with open(MC_PATH, "rb") as pf:
        mc_lca = pickle.load(pf)

mc_lca_bkp = copy.deepcopy(mc_lca)

In [ ]:
df_mc_lca = dict()
for j, sce_name in enumerate(mc_lca.keys()):
    df_mc_lca[sce_name] = pd.DataFrame.from_dict(mc_lca[sce_name])
    df_mc_lca[sce_name].columns = all_methods_names

for sce in SCENARIOS:
    print(sce)
    display(df_mc_lca[sce])

if RUN_MC or not os.path.exists(MC_PATH):
    with open(MC_PATH, "wb") as fp:
        pickle.dump(df_mc_lca, fp)

In [ ]:
if RUN_MC or not os.path.exists(MC_PATH_LCA_ONLY):
    results_lca_dict = dict()

    for sce_name, sce in zip(
        SCENARIOS, list_of_NgScenarioLCA
    ):
        sce.doLCA(list(dict_ipcc_methods.values()))
        results_lca_dict[sce_name] = {k: v.score / 1e12 for k, v in zip(all_methods_names, sce.lca_results)}

    with open(MC_PATH_LCA_ONLY, "wb") as fp:
        pickle.dump(results_lca_dict, fp)
else:
    with open(MC_PATH_LCA_ONLY, "rb") as fp:
        results_lca_dict = pickle.load(fp)

df_results_lca_only = pd.DataFrame.from_dict(results_lca_dict).T

In [ ]:
# Basic statistical results from MC simulations
import scipy


dict_stats_mc = {}
df_results_lca_only

for j, (i, v) in enumerate(df_mc_lca.items()):
    if i not in SCENARIOS:
        continue

    q_quantile = 0.01
    v_no_outlier = {}
    for col in v.columns:
        q_low = v[col].quantile(q_quantile)
        q_hi = v[col].quantile(1 - q_quantile)
        v_no_outlier[col] = v[(v[col] <= q_hi) & (v[col] >= q_low)][col]

    v = pd.DataFrame(v_no_outlier)

    df_base_mc_results = v.describe().T
    df_base_mc_results[[col.capitalize() for col in df_base_mc_results.columns]] = (
        df_base_mc_results
    )
    df_base_mc_results["Baseline"] = df_results_lca_only.loc[i]

    df_base_mc_results["Mean-Baseline diff. [%]"] = (
        100
        * (df_base_mc_results["Baseline"] - df_base_mc_results["Mean"])
        / df_base_mc_results["Baseline"]
    )

    distance_from_mean = (df_base_mc_results["75%"] - df_base_mc_results["25%"]) / 2

    df_base_mc_results["CV [%]"] = 100 * distance_from_mean / df_base_mc_results["Mean"]
    df_base_mc_results["CV-Std [%]"] = (
        100 * df_base_mc_results["Std"] / df_base_mc_results["Mean"]
    )
    df_base_mc_results["QCD [%]"] = (
        100
        * abs(df_base_mc_results["75%"] - df_base_mc_results["25%"])
        / abs(df_base_mc_results["75%"] + df_base_mc_results["25%"])
    )

    df_base_mc_results["MAD [%]"] = (
        100 * scipy.stats.median_abs_deviation(df_mc_lca[i]) / df_mc_lca[i].mean()
    )

    df_base_mc_results = df_base_mc_results[
        [
            "Baseline",
            "Mean",
            "Std",
            "QCD [%]",
            "Min",
            "25%",
            "50%",
            "75%",
            "Max",
        ]
    ]

    dict_stats_mc[i] = df_base_mc_results

    df_base_mc_results_fmt = df_base_mc_results.map("{:,.2e}".format)
    df_base_mc_results_fmt["QCD [%]"] = pd.to_numeric(
        df_base_mc_results_fmt["QCD [%]"]
    ).map("{:,.2f}".format)

    print(i + ":")
    display(
        df_base_mc_results_fmt.loc[df_base_mc_results.sort_values(by="QCD [%]").index]
    )

    if save_tables:
        with pd.ExcelWriter(
            "./data/results/lca_IPCC_MC_statistical_results.xlsx",
            engine="openpyxl",
            mode="w" if j == 0 else "a",
        ) as writer:
            df_base_mc_results_fmt.to_excel(writer, sheet_name=f"MC-IPCC-{i}")

#### Uncertainty error bars

In [ ]:
fig, ax = plot_ipcc_breakdown(
    scenarios_ipcc_repower,
    scenario_names_repower,
    color_scenario_repower,
    dict_stats_mc=dict_stats_mc,
    save_fig=save_fig,
    path2save="./figs/delta_repower_only_errorbars",
)

ipd.clear_output()
plt.show()

In [ ]:
fig, ax = plot_ipcc_breakdown(
    scenarios_ipcc_y2022[:-1],
    scenario_names_y2022,
    color_scenario_y2022,
    dict_stats_mc=dict_stats_mc,
    save_fig=save_fig,
    path2save="./figs/delta_y2022_only_errorbars",
)

ipd.clear_output()
plt.show()

In [ ]:
fig, axs_ = plt.subplot_mosaic(
    """
    AAABBB
    """,
    figsize=(fig_length[2], 0.3 * fig_height),
    dpi=300 if save_fig else 100,
)
axs = np.array([axs_[key] for key in sorted(axs_.keys())])

fig, axs = plot_ipcc_breakdown(
    scenarios_ipcc_coal[:-1],
    scenario_names_coal,
    color_scenario_coal,
    dict_stats_mc=dict_stats_mc,
    save_fig=save_fig,
    axs=axs,
    path2save="./figs/delta_coal_only_errorbars",
)

# axis manipulation
ylo, yhi = 0.6, 1.4
for ax in axs:
    ax.set_ylim((ylo, yhi))
    # add a white box in y=0.65
    axs0_lim = list(ax.get_xlim())
    ax.fill_between(
        axs0_lim, ylo + 0.0405, ylo + 0.0585, color="white", zorder=3, linewidth=0.0
    )
    # remove minor tick
    ax.minorticks_off()
    ax.yaxis.set_ticks([ylo + 0.04, ylo + 0.06], minor=True)
    ax.yaxis.set_tick_params(which="minor", direction="inout", length=4, rotation=45)
    new_yticks = [0.0] + [round(ylo + 0.1 * (1 + plus), 1) for plus in range(7)]
    ax.yaxis.set_ticklabels([str(val) for val in new_yticks])


for label, ax in zip(string.ascii_uppercase, axs):
    ax.annotate(
        label + ")",
        xy=(0, 1),
        xycoords="axes fraction",
        xytext=(+0.5, -0.5),
        textcoords="offset fontsize",
        fontsize="medium",
        verticalalignment="top",
        fontfamily="calibri",
    )

plt.tight_layout()

if save_fig:
    plt.savefig("./figs/delta_coal_only_errorbars.svg", bbox_inches="tight")

ipd.clear_output()
plt.show()

In [ ]:
fig, axs_ = plt.subplot_mosaic(
    """
    AAABBB
    """,
    figsize=(fig_length[2], 0.3 * fig_height),
    dpi=300 if save_fig else 100,
)
axs = np.array([axs_[key] for key in sorted(axs_.keys())])

fig, axs = plot_ipcc_breakdown(
    scenarios_ipcc_clean,
    scenario_names_clean,
    color_scenario_clean,
    dict_stats_mc=dict_stats_mc,
    save_fig=save_fig,
    axs=axs,
    path2save="./figs/delta_clean_only_errorbars",
)
# axis manipulation
ylo, yhi = 0.6, 1.4
for ax in axs:
    ax.set_ylim((ylo, yhi))
    # add a white box in y=0.65
    axs0_lim = list(ax.get_xlim())
    ax.fill_between(
        axs0_lim, ylo + 0.0405, ylo + 0.0585, color="white", zorder=3, linewidth=0.0
    )
    # remove minor tick
    ax.minorticks_off()
    ax.yaxis.set_ticks([ylo + 0.04, ylo + 0.06], minor=True)
    ax.yaxis.set_tick_params(which="minor", direction="inout", length=4, rotation=45)
    new_yticks = [0.0] + [round(ylo + 0.1 * (1 + plus), 1) for plus in range(7)]
    ax.yaxis.set_ticklabels([str(val) for val in new_yticks])

for label, ax in zip(string.ascii_uppercase, axs):
    ax.annotate(
        label + ")",
        xy=(0, 1),
        xycoords="axes fraction",
        xytext=(+0.5, -0.5),
        textcoords="offset fontsize",
        fontsize="medium",
        verticalalignment="top",
        fontfamily="calibri",
        # bbox=dict(facecolor="0.7", edgecolor="none", pad=3.0),
    )

plt.tight_layout()

if save_fig:
    plt.savefig("./figs/delta_clean_only_errorbars.svg", bbox_inches="tight")

ipd.clear_output()
plt.show()

In [ ]:
# create dummy axis
fig_dummy, ax_dummy = plt.subplots()
fig, axs_ = plt.subplot_mosaic(
    """
    AAABBB
    """,
    figsize=(fig_length[2], 0.3 * fig_height),
    dpi=300 if save_fig else 100,
)
axs = np.array([axs_[key] for key in sorted(axs_.keys())])

plot_ipcc_breakdown(
    scenarios_ipcc_repower,
    scenario_names_repower,
    color_scenario_repower,
    dict_stats_mc=dict_stats_mc,
    axs=[ax_dummy, axs[0]],
)
plot_ipcc_breakdown(
    scenarios_ipcc_y2022[:-1],
    scenario_names_y2022,
    color_scenario_y2022,
    dict_stats_mc=dict_stats_mc,
    axs=[ax_dummy, axs[1]],
)

# axis manipulation
ylo, yhi = 0.6, 1.4
for ax in axs[0:2]:
    ax.set_ylim((ylo, yhi))
    # add a white box in y=0.65
    axs0_lim = list(ax.get_xlim())
    ax.fill_between(
        axs0_lim, ylo + 0.0405, ylo + 0.0585, color="white", zorder=3, linewidth=0.0
    )
    # remove minor tick
    ax.minorticks_off()
    ax.yaxis.set_ticks([ylo + 0.04, ylo + 0.06], minor=True)
    ax.yaxis.set_tick_params(which="minor", direction="inout", length=4, rotation=45)
    new_yticks = [0.0] + [round(ylo + 0.1 * (1 + plus), 1) for plus in range(7)]
    ax.yaxis.set_ticklabels([str(val) for val in new_yticks])


for label, ax in axs_.items():
    ax.annotate(
        label+")",
        xy=(0, 1),
        xycoords="axes fraction",
        xytext=(+0.5, -0.5),
        textcoords="offset fontsize",
        fontsize="medium",
        verticalalignment="top",
        fontfamily="calibri",
        # bbox=dict(facecolor="0.7", edgecolor="none", pad=3.0),
    )

# close fig_dummy
plt.close(fig_dummy)

ipd.clear_output()
plt.tight_layout()

if save_fig:
    plt.savefig("./figs/fig2_ESI.svg")
    plt.savefig("./figs/fig2_ESI.png", dpi=300)

#### Climate change distribution

In [ ]:
fig_ipcc_dist, ax_ipcc_dist = plot_cc_hist(df_mc_lca, save_fig=save_fig, N_mc=N_mc)

plt.show()

In [ ]:
fig_ipcc_dist, ax_ipcc_dist = plot_cc_violin(df_mc_lca, save_fig=save_fig, N_mc=N_mc)

plt.show()

In [ ]:
print("Comparison between constant per capita budget and fixed yearly budget")
plot_cc_violin(
    {k: df_mc_lca[k] for k in SCENARIOS[0:5]},
    budget15=cum_budget_1_5C * EU_POPULATION / CUM_GLOBAL_POPULATION,
    budget20=cum_budget_2C * EU_POPULATION / CUM_GLOBAL_POPULATION,
)
fig, ax = plot_cc_violin(
    {k: df_mc_lca[k] for k in SCENARIOS[0:5]},
    budget15=cum_budget_1_5C / 80 * (EU_POPULATION / GLOBAL_POPULATION),
    budget20=cum_budget_2C / 80 * (EU_POPULATION / GLOBAL_POPULATION),
)

plt.show()

if save_fig:
    fig.savefig(
        f"./figs/mc_violin-box_cc_{N_mc}_ite_fixedYearlyBudget.svg", bbox_inches="tight"
    )

In [ ]:
# Life cycle CO2 emissions coefficient of variation
q_quantile = 0.0
v_no_outlier = {}
category = "IPCC 2021 | Life cycle CO2 emissions"

for j, sce_name in enumerate(df_mc_lca.keys()):
    df_mc_lca_tmp = df_mc_lca[sce_name][category]
    q_low = df_mc_lca_tmp.quantile(q_quantile)
    q_hi = df_mc_lca_tmp.quantile(1 - q_quantile)
    v_no_outlier[sce_name] = df_mc_lca_tmp.loc[
        (df_mc_lca_tmp <= q_hi) & (df_mc_lca_tmp >= q_low)
    ]

df_mc_lca_no_outlier = pd.DataFrame(v_no_outlier)

df_mc_lca_no_outlier_tmp = []
for col in df_mc_lca_no_outlier.columns:
    df_mc_lca_no_outlier_tmp.append(
        df_mc_lca_no_outlier[[col]].rename(columns={col: col})
    )

df_box_plot_cc = pd.concat(df_mc_lca_no_outlier_tmp, axis=1)
df_box_plot_cc = 100 * df_box_plot_cc / (501 * 447.0e6)

desc = df_box_plot_cc.describe()

display(
    Latex(
        r"Life cycle CO2 emissions - Coefficient of variation $\left(\frac{\sigma}{\mu}\right)$ [%]"
    )
)
100 * desc.loc["std"] / desc.loc["mean"]

#### P(A > B) visualizations

##### Paired t-test for identical means

1. Calculate the difference between the paired observations: $d = x_1 - x_2$, where $x_1$ and $x_2$ are the paired observations.
2. Calculate the mean of the differences: $\bar{d} = \frac{1}{n}\sum_{i=1}^{n} d_i$, where $n$ is the number of paired observations.
3. Calculate the standard deviation of the differences: $s_d = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(d_i - \bar{d})^2}$.
4. Calculate the standard error of the mean difference: $SE = \frac{s_d}{\sqrt{n}}$.
5. Calculate the t-statistic: $t = \frac{\bar{d}}{SE}$.
6. Determine the degrees of freedom: $df = n - 1$.
7. Calculate the p-value using the t-distribution with the appropriate degrees of freedom.

The p-value represents the probability of observing a t-statistic as extreme as the one calculated, assuming the null hypothesis is true (i.e., the means are equal). A small p-value indicates strong evidence against the null hypothesis, suggesting that the means are significantly different.

##### Probability of superiority $P(A>B)$

$$P(A>B) = \sum_{i=1}^n\frac{\min\{0,\max\{1,a_i-b_i\}\}}{n}
$$

In [ ]:
def create_prob_mc_df(base):
    prob_dict = {}
    prob_numeric_dict = {}

    for col in df_mc_lca[base].columns:
        scenario_dict = {}
        a = df_mc_lca[base][col]

        for sce in SCENARIOS:
            if sce == base:
                continue
            b = df_mc_lca[sce][col]
            correlation, p_value = pearsonr(a, b)
            t_test_rel = scipy.stats.ttest_rel(a, b)
            z_test = round(t_test_rel.pvalue, 4)

            scenario_dict[f"P[{base} > {sce}]"] = (a > b).mean()
            scenario_dict[f"{base} == {sce}?"] = z_test
            

        prob_numeric_dict[col] = scenario_dict

    return pd.DataFrame.from_dict(prob_numeric_dict).T

df_prob_mc_numeric = create_prob_mc_df(SCENARIOS[0])
df_prob_mc_numeric_repower = create_prob_mc_df(SCENARIOS[1])
df_prob_mc_numeric_y2022 = create_prob_mc_df(SCENARIOS[2])
df_prob_mc_numeric

In [ ]:
def create_A_minus_B_df(base):
    base_minus_scenario = {}

    for sce in SCENARIOS:
        base_minus_scenario[sce] = (
            100 * (df_mc_lca[sce] - df_mc_lca[base]) / df_mc_lca[base]
        )

        for col in base_minus_scenario[sce].columns:
            base_minus_scenario[sce][f"color_{col}"] = -1
            base_minus_scenario[sce].loc[
                base_minus_scenario[sce][col] > 0, f"color_{col}"
            ] = 1

        base_minus_scenario[sce]["dummy_color"] = 0

        base_minus_scenario[sce].rename(
            columns={
                col: col.replace("_", " ")
                .replace("color ", "color_")
                .replace("dummy ", "dummy_")
                for col in base_minus_scenario[sce].columns
            },
            inplace=True,
        )
    return base_minus_scenario

base_minus_scenario = create_A_minus_B_df(SCENARIOS[0])
repower_minus_scenario = create_A_minus_B_df(SCENARIOS[1])
y2022_minus_scenario = create_A_minus_B_df(SCENARIOS[2])

In [ ]:
fig_ipcc_a_minus_b, ax_ipcc_a_minus_b = plot_cc_hist_comparison(
    base_minus_scenario[SCENARIOS[1]], df_prob_mc_numeric, save_fig=save_fig, N_mc=N_mc
)

plt.show()

In [ ]:
fig_ipcc_a_minus_b, ax_ipcc_a_minus_b = plot_cc_hist_comparison(
    base_minus_scenario[SCENARIOS[2]],
    df_prob_mc_numeric,
    scenario=SCENARIOS[2],
    save_fig=save_fig,
    N_mc=N_mc,
)

plt.show()

In [ ]:
fig_ipcc_a_minus_b, ax_ipcc_a_minus_b = plot_cc_hist_comparison(
    repower_minus_scenario[SCENARIOS[2]],
    df_prob_mc_numeric_repower,
    base=SCENARIOS[1],
    scenario=SCENARIOS[2],
    save_fig=save_fig,
    N_mc=N_mc,
)

plt.show()

## Combining figures

In [ ]:
fig_length = {
    1: 3.50394,  # 1 column
    1.5: 5.35433,  # 1.5 columns
    2: 7.20472,
}  # 2 columns
fig_height = 9.72441  # maxium height
fontsize_title = 9 - 0
fontsize_label = 8 - 0
fontsize_legend = 8 - 0
fontsize_axs = 8 - 0

spineline_width = 0.3

plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.linewidth"] = spineline_width

plt.rcParams["font.family"] = "calibri"  # "times new roman"
plt.rcParams["legend.facecolor"] = "white"
plt.rcParams["legend.edgecolor"] = "black"
plt.rcParams["legend.shadow"] = False
plt.rcParams["legend.facecolor"] = "white"
plt.rcParams["legend.edgecolor"] = "black"
plt.rcParams["font.size"] = fontsize_axs
plt.rcParams["legend.fontsize"] = fontsize_legend - 2
plt.rcParams["axes.labelsize"] = fontsize_axs
plt.rcParams["ytick.labelsize"] = fontsize_axs
plt.rcParams["xtick.labelsize"] = fontsize_axs
plt.rcParams["axes.labelpad"] = 0.0
plt.rcParams["axes.linewidth"] = spineline_width
plt.rcParams["axes.spines.bottom"] = True
plt.rcParams["axes.spines.left"] = True
plt.rcParams["axes.spines.right"] = True
plt.rcParams["axes.spines.top"] = True
plt.rcParams["axes.titlesize"] = fontsize_label
plt.rcParams["xtick.labelsize"] = fontsize_label
plt.rcParams["xtick.major.width"] = spineline_width
plt.rcParams["ytick.major.width"] = spineline_width
plt.rcParams["xtick.minor.width"] = spineline_width
plt.rcParams["ytick.minor.width"] = spineline_width
plt.rcParams["figure.titlesize"] = fontsize_title
plt.rcParams["grid.linewidth"] = spineline_width
plt.rcParams["axes.grid.axis"] = "y"

# create dummy axis
fig_dummy, ax_dummy = plt.subplots()
fig, axs = plt.subplots(
    2,
    3,
    figsize=(1.12 * fig_length[2], 0.55 * fig_height),
    dpi=300 if save_fig else 100,
)
axs = axs.flatten()

plot_ipcc_breakdown(
    scenarios_ipcc_repower,
    scenario_names_repower,
    color_scenario_repower,
    dict_stats_mc=dict_stats_mc,
    axs=[axs[0], ax_dummy],
)
plot_ipcc_breakdown(
    scenarios_ipcc_y2022[:-1],
    scenario_names_y2022,
    color_scenario_y2022,
    dict_stats_mc=dict_stats_mc,
    axs=[axs[1], ax_dummy],
)

# axis manipulation
ylo, yhi = 0.7, 1.2
for ax in axs[0:2]:
    ax.set_ylim((ylo, yhi))
    # add a white box in y=0.65
    axs0_lim = list(ax.get_xlim())
    ax.fill_between(
        axs0_lim, ylo + 0.0405, ylo + 0.0585, color="white", zorder=3, linewidth=0.0
    )
    # remove minor tick
    ax.minorticks_off()
    ax.yaxis.set_ticks([ylo + 0.04, ylo + 0.06], minor=True)
    ax.yaxis.set_tick_params(which="minor", direction="inout", length=4, rotation=45)
    new_yticks = [0.0] + [round(ylo + 0.1 * (1 + plus), 1) for plus in range(7)]
    ax.yaxis.set_ticklabels([str(val) for val in new_yticks])

plot_cc_violin({k: df_mc_lca[k] for k in SCENARIOS[0:3]}, fig=fig, ax=axs[2])

plot_cc_hist_comparison(
    base_minus_scenario[SCENARIOS[1]], df_prob_mc_numeric, fig=fig, ax=axs[3]
)
plot_cc_hist_comparison(
    base_minus_scenario[SCENARIOS[2]],
    df_prob_mc_numeric,
    scenario=SCENARIOS[2],
    fig=fig,
    ax=axs[4],
)
plot_cc_hist_comparison(
    y2022_minus_scenario[SCENARIOS[1]],
    df_prob_mc_numeric_y2022,
    base=SCENARIOS[2],
    scenario=SCENARIOS[1],
    fig=fig,
    ax=axs[5],
)


plt.close(fig_dummy)

ipd.clear_output()

plt.tight_layout()
plt.show()

In [ ]:
# create dummy axis
fig_dummy, ax_dummy = plt.subplots()
fig, axs_ = plt.subplot_mosaic(
    """
    AAABBB
    CCDDEE
    """,
    figsize=(1.12 * fig_length[2], 0.55 * fig_height),
    dpi=300 if save_fig else 100,
)
axs = np.array([axs_[key] for key in sorted(axs_.keys())])

plot_ipcc_breakdown(
    scenarios_ipcc_repower,
    scenario_names_repower,
    color_scenario_repower,
    dict_stats_mc=dict_stats_mc,
    axs=[axs[0], ax_dummy],
)
plot_ipcc_breakdown(
    scenarios_ipcc_y2022[:-1],
    scenario_names_y2022,
    color_scenario_y2022,
    dict_stats_mc=dict_stats_mc,
    axs=[axs[1], ax_dummy],
)

# axis manipulation
ylo, yhi = 0.7, 1.2
for ax in axs[0:2]:
    ax.set_ylim((ylo, yhi))
    # add a white box in y=0.65
    axs0_lim = list(ax.get_xlim())
    ax.fill_between(
        axs0_lim, ylo + 0.0405, ylo + 0.0585, color="white", zorder=3, linewidth=0.0
    )
    # remove minor tick
    ax.minorticks_off()
    ax.yaxis.set_ticks([ylo + 0.04, ylo + 0.06], minor=True)
    ax.yaxis.set_tick_params(which="minor", direction="inout", length=4, rotation=45)
    new_yticks = [0.0] + [round(ylo + 0.1 * (1 + plus), 1) for plus in range(7)]
    ax.yaxis.set_ticklabels([str(val) for val in new_yticks])

plot_cc_violin({k: df_mc_lca[k] for k in SCENARIOS[0:3]}, fig=fig, ax=axs[4])

plot_cc_hist_comparison(
    base_minus_scenario[SCENARIOS[1]], df_prob_mc_numeric, fig=fig, ax=axs[2]
)
plot_cc_hist_comparison(
    base_minus_scenario[SCENARIOS[2]],
    df_prob_mc_numeric,
    scenario=SCENARIOS[2],
    fig=fig,
    ax=axs[3],
)

for label, ax in axs_.items():
    ax.annotate(
        label+")",
        xy=(0, 1),
        xycoords="axes fraction",
        xytext=(+0.5, -0.5),
        textcoords="offset fontsize",
        fontsize="medium",
        verticalalignment="top",
        fontfamily="calibri",
        # bbox=dict(facecolor="0.7", edgecolor="none", pad=3.0),
    )

# close fig_dummy
plt.close(fig_dummy)

ipd.clear_output()

plt.tight_layout()
plt.show()

if save_fig:
    plt.savefig("./figs/fig2_merged.svg")
    plt.savefig("./figs/fig2_merged.png", dpi=300)